Dual energy carbonate CT
========================

test


In [1]:
import ipyparallel as ipp

# Create an MPI cluster with 8 engines
cluster = ipp.Cluster(engines="mpi", n=40)

# Start and connect to the cluster
rc = cluster.start_and_connect_sync()

# Enable IPython magics for parallel processing
rc[:].activate()

Starting 40 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/40 [00:00<?, ?engine/s]

In [ ]:
%%px --block
import rockverse as rv
#rv.selected_gpus = []
dectgroup = rv.open('/estgf_dados/P_D/GOB7/testemunhos/C04B21_DGX')
dectgroup.run(restart=True)

[stdout:0] [2025-02-20 15:47:18] Hashing Low attenuation: 100% 16/16 [00:00<00:00, 122.21chunk/s]
[2025-02-20 15:47:18] Hashing High attenuation: 100% 16/16 [00:00<00:00, 114.69chunk/s]
[2025-02-20 15:47:18] Hashing mask: 100% 16/16 [00:00<00:00, 332.61chunk/s]
[2025-02-20 15:47:18] Hashing segmentation: 100% 16/16 [00:00<00:00, 327.53chunk/s]
[2025-02-20 15:47:22] Generating inversion coefficients: 100% 100000/100000 [00:03<00:00, 30858.02/s]
[2025-02-20 15:47:26] Creating output images: 11it [00:03,  3.42it/s]
[2025-02-20 15:47:29] Counting voxels: 100% 16/16 [00:02<00:00,  7.99chunk/s]
[2025-02-20 15:47:31] rho/Z inversion (chunk 1/16):   0% 26684/39445817 [01:19<26:43:22, 409.75voxel/s]

%px:   0%|          | 0/40 [00:00<?, ?tasks/s]

In [ ]:
# RockVerse will normalize the pdf values before assigning them.
# Let's check that also (again, make sure all processes run these four lines)
x_lowE_pdf, y_lowE_pdf = dectgroup.calibration_material[cm_index].lowE_pdf
x_highE_pdf, y_highE_pdf = dectgroup.calibration_material[cm_index].highE_pdf

#Cumulative density function (CDF) is also available
_, cdf_low = dectgroup.calibration_material[cm_index].lowE_cdf
_, cdf_high = dectgroup.calibration_material[cm_index].highE_cdf

if rv.mpi_rank == 0:
    fig, ax = plt.subplots(1, 2, figsize=(9, 4.5),
                            layout='constrained',
                            sharex=True, sharey=True)
    fig.suptitle('Probability density functions')

    for k, (x, y, c, E) in enumerate(zip((x_lowE_pdf, x_highE_pdf),
                                         (y_lowE_pdf, y_highE_pdf),
                                         (cdf_low, cdf_high),
                                         ('Low', 'High'))):
        ax[k].plot(x, y, color='forestgreen')
        ax[k].set_xlabel(f'{E} energy Air CT (HU)')
        ax[k].set_ylabel('PDF model', color='forestgreen')
        ax_cdf = ax[k].twinx()
        ax_cdf.plot(x, c, color='tomato')
        ax_cdf.set_ylabel('CDF model', color='tomato')
        ax_cdf.set_ylim(ymin=0)
    ax[0].set_ylim(ymin=0)


In [ ]:
%%px --block

seg_phase = 2 # Water is segmentation phase 1
cm_index = 1  # We'll assign water to calibration material 1
name = 'Water'
bulk_density = 1.
composition = {'H': 2, 'O': 1}

x_low = lowE_viewer.histogram.bin_centers
y_low = lowE_viewer.histogram.count[seg_phase].values
x_high = highE_viewer.histogram.bin_centers
y_high = highE_viewer.histogram.count[seg_phase].values

# Call gaussian_fit to get fitting parameters
c_low = gaussian_fit(x_low, y_low)
c_high = gaussian_fit(x_high, y_high)

# Build a fine-spaced histogram axis and call
# gaussian_val to build the gaussian curve.
# It is important sample regions
# with PDF values close to zero
xlim = (-200, 150)
x_fit = np.linspace(*xlim, 200)
y_fit_low = gaussian_val(c_low, x_fit)
y_fit_high = gaussian_val(c_high, x_fit)

# Let's check the Gaussian fit:
if mpi_rank == 0:
    fig, ax = plt.subplots(1, 2, figsize=(10, 5),
                           layout='constrained',
                           sharex=True, sharey=True)
    fig.suptitle('Gaussian histogram fit')

    for k, (x, y, yfit, E) in enumerate(zip((x_low, x_high),
                                            (y_low, y_high),
                                            (y_fit_low, y_fit_high),
                                            ('Low', 'High'))):

        ax[k].plot(x, y, '.--', label='Histogram')
        ax[k].plot(x_fit, yfit, '-', label='Gaussian fit')
        ax[k].set_xlabel(f'{name} {E} energy CT (HU)')
        ax[k].set_xlim(xlim)
        ax[k].legend()
        ax[k].set_ylabel('Count')
    ax[0].set_ylim(ymin=0)

# Now fill in necessary fields in dectgroup
# Note: setting or getting PDFs is a collective MPI call,
# make sure all the processes run it (e.g. don't use if 'mpi_rank == 0')
dectgroup.calibration_material[cm_index].description = name
dectgroup.calibration_material[cm_index].lowE_pdf = (x_fit, y_fit_low)
dectgroup.calibration_material[cm_index].highE_pdf = (x_fit, y_fit_high)
dectgroup.calibration_material[cm_index].bulk_density = bulk_density
dectgroup.calibration_material[cm_index].composition = composition

# RockVerse will normalize the PDF values before assigning them.
# Let's check that also (again, make sure all processes run these four lines)
x_lowE_pdf, y_lowE_pdf = dectgroup.calibration_material[cm_index].lowE_pdf
_, cdf_low = dectgroup.calibration_material[cm_index].lowE_cdf
x_highE_pdf, y_highE_pdf = dectgroup.calibration_material[cm_index].highE_pdf
_, cdf_high = dectgroup.calibration_material[cm_index].highE_cdf

if rv.mpi_rank == 0:
    fig, ax = plt.subplots(1, 2, figsize=(9, 4.5),
                           layout='constrained',
                           sharex=True, sharey=True)
    fig.suptitle('Probability density functions')

    for k, (x, y, c, E) in enumerate(zip((x_lowE_pdf, x_highE_pdf),
                                         (y_lowE_pdf, y_highE_pdf),
                                         (cdf_low, cdf_high),
                                         ('Low', 'High'))):
        ax[k].plot(x, y, color='forestgreen')
        ax[k].set_xlabel(f'{E} energy Air CT (HU)')
        ax[k].set_ylabel('PDF model', color='forestgreen')
        ax_cdf = ax[k].twinx()
        ax_cdf.plot(x, c, color='tomato')
        ax_cdf.set_ylabel('CDF model', color='tomato')
        ax_cdf.set_ylim(ymin=0)
    ax[0].set_ylim(ymin=0)


In [ ]:


# RockVerse will normalize the PDF values before assigning them.
# Let's check that also (again, make sure all processes run these four lines)
x_lowE_pdf, y_lowE_pdf = dectgroup.calibration_material[cm_index].lowE_pdf
x_highE_pdf, y_highE_pdf = dectgroup.calibration_material[cm_index].highE_pdf
_, cdf_low = dectgroup.calibration_material[cm_index].lowE_cdf
_, cdf_high = dectgroup.calibration_material[cm_index].highE_cdf

if rv.mpi_rank == 0:
    fig, ax = plt.subplots(1, 2, figsize=(9, 4.5),
                           layout='constrained',
                           sharex=True, sharey=True)
    fig.suptitle('Probability density functions')

    for k, (x, y, c, E) in enumerate(zip((x_lowE_pdf, x_highE_pdf),
                                         (y_lowE_pdf, y_highE_pdf),
                                         (cdf_low, cdf_high),
                                         ('Low', 'High'))):
        ax[k].plot(x, y, color='forestgreen')
        ax[k].set_xlabel(f'{E} energy Air CT (HU)')
        ax[k].set_ylabel('PDF model', color='forestgreen')
        ax_cdf = ax[k].twinx()
        ax_cdf.plot(x, c, color='tomato')
        ax_cdf.set_ylabel('CDF model', color='tomato')
        ax_cdf.set_ylim(ymin=0)
    ax[0].set_ylim(ymin=0)


In [ ]:

# RockVerse will normalize the PDF values before assigning them.
# Let's check that also (again, make sure all processes run these four lines)
x_lowE_pdf, y_lowE_pdf = dectgroup.calibration_material[cm_index].lowE_pdf
x_highE_pdf, y_highE_pdf = dectgroup.calibration_material[cm_index].highE_pdf
_, cdf_low = dectgroup.calibration_material[cm_index].lowE_cdf
_, cdf_high = dectgroup.calibration_material[cm_index].highE_cdf

if rv.mpi_rank == 0:
    fig, ax = plt.subplots(1, 2, figsize=(9, 4.5),
                           layout='constrained',
                           sharex=True, sharey=True)
    fig.suptitle('Probability density functions')

    for k, (x, y, c, E) in enumerate(zip((x_lowE_pdf, x_highE_pdf),
                                         (y_lowE_pdf, y_highE_pdf),
                                         (cdf_low, cdf_high),
                                         ('Low', 'High'))):
        ax[k].plot(x, y, color='forestgreen')
        ax[k].set_xlabel(f'{E} energy Air CT (HU)')
        ax[k].set_ylabel('PDF model', color='forestgreen')
        ax_cdf = ax[k].twinx()
        ax_cdf.plot(x, c, color='tomato')
        ax_cdf.set_ylabel('CDF model', color='tomato')
        ax_cdf.set_ylim(ymin=0)
    ax[0].set_ylim(ymin=0)